In [701]:
# Reset and import packages
%reset -fs 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [ ]:
engine = create_engine("sqlite:///mta2019.db")
df = pd.read_sql("SELECT * \
                  FROM mta_data_spring \
                  WHERE desc = 'REGULAR' AND time LIKE '%00' \
                  UNION \
                  SELECT * \
                  FROM mta_data_summer \
                  WHERE desc = 'REGULAR' AND time LIKE '%00'", engine)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# Renaming columns
df.columns = ['index', 'c_a', 'unit', 'scp', 'station', 'linename', 'division', 'date', 'time', 'desc', 'entries', 'exits']
df.insert(0, 'identifier', df["c_a"] + "|" + df["unit"] + "|" + df["scp"]) # Cleaning up the columns
del df["index"]
del [df["c_a"], df["unit"], df["scp"]]

In [ ]:
# Combining date + time to datetime object

df.insert(4, 'datetime', df['date'] + df['time'])
del [df['time']]
df['datetime'] = pd.to_datetime(df['datetime'], format = "%m/%d/%Y%H:%M:%S")


df.head()

In [ ]:
df_sorted = df.sort_values(by = ['identifier','datetime']) # sorting to make sure that each turnstile is grouped, the sorted by datetime

In [ ]:
df_sorted['new_entries'] = df_sorted['entries'].diff() # creation of new_entries and new_exits
df_sorted['new_exits'] = df_sorted['exits'].diff()
df_sorted.head()

In [ ]:
df_sorted = df_sorted[(df_sorted['identifier'] == df_sorted.shift(1)['identifier'])] # making sure no turnstile is being compared to a different
                                                                                     # turnstile for new_entries and new_exits


In [ ]:
df_sorted['new_entries'] = df_sorted['new_entries'].abs() # flipping all values to be positive
df_sorted['new_exits'] = df_sorted['new_exits'].abs()
df_sorted['new_entries'].describe()

In [ ]:
#Figure 2: Cleaning the data - histogram shows values that are within normal range

plt.figure(figsize = (12,9))
plt.hist(np.log10(df_sorted["new_entries"].abs() + 1), bins = 50) # visualizing distribution of values on a logrithmic scale
plt.title("Frequency distribution of new entries")
plt.xlabel("10^n new entries")
plt.ylabel("Frequency")

plt.xlim(0,4)
plt.savefig(fname = "Figure 2.png")

In [ ]:
df_sorted = df_sorted.loc[~(df_sorted['new_entries'].abs() > 10 ** 3.8)] # getting rid of values > 10 ^ 3.8
df_sorted = df_sorted.loc[~(df_sorted['new_exits'].abs() > 10 ** 3.8)]

df_sorted_no_outliers = df_sorted.reset_index(drop = True) # resetting the index
df_sorted_no_outliers.sort_values(by = 'new_exits', ascending = False).head()

In [ ]:
df_clean = df_sorted_no_outliers.groupby(by = ["station", "datetime", "linename"], as_index = False)[['new_entries', 'new_exits']].sum()
                                                                            
df_clean['total_traffic'] = df_clean['new_entries'] + df_clean['new_exits']
df_clean['trains_per_h'] = list(zip(df_clean['datetime'], df_clean['linename']))

In [ ]:
df_clean.insert(2, 'dow', df_clean['datetime']) # Making a column that only holds day of week and time
df_clean['dow'] = df_clean['dow'].apply(lambda x: x.weekday())
df_clean['dow'] = df_clean['dow'].map({0: "M", # Mapping day of week
                     1: "Tu",
                     2: "W",
                     3: "Th",
                     4: "F",
                     5: "Sa",
                     6: "Su"})
df_clean.insert(3, 'time', df_clean['datetime']) 
df_clean['time'] = df_clean['time'].apply(lambda x: x.time()) # Getting time from datetime
df_clean.insert(4, 'daytime', df_clean['dow'].str.cat(df_clean['time'].apply(str), sep = " ")) # Giving column the format "dow, time"
df_clean.drop(columns = ['dow', 'time'], inplace = True)

In [ ]:
frequency_table = pd.read_csv("NYC Subway - Sheet1.csv", index_col = 0).T # Reading in the frequency table and transposing it
frequency_table.rename({"Period":"Line"}, inplace = True) # Cleaning up the frequency table
frequency_table = 60 / frequency_table # Converting from minutes/train to trains/hour
frequency_table.fillna(0, inplace = True) # Getting rid of NAs
frequency_table = frequency_table.round(2) # Making it more readeable
frequency_table.loc['Z'] = frequency_table.loc["J/Z"] # Splitting the J and Z lines
frequency_table.rename(index = {"J/Z": "J"}, inplace = True)
frequency_table

In [ ]:
#set(df_clean['linename'].apply(list).sum()) # Checking to make sure I have all the lines accounted for before apply()

In [ ]:
# deprecated method to determine train frequency
# def trains_per_h_conv(date_and_line): # Takes in a tuple (datetime, line), outputs the frequency of trains during that period
#     dt_value = date_and_line[0]
#     line = date_and_line[1]
#     trains = 0
#     if dt_value.time() < datetime(1,1,1, 6, 30, 0).time(): # Determining the time of day it falls under
#         code = "Late nights"
#     elif dt_value.weekday() > 4:
#         code = "Weekends"
#     elif dt_value.time() > datetime(1,1,1, 20, 0, 0).time():
#         code = "Evenings"
#     elif dt_value.time() > datetime(1,1,1, 15, 30, 0).time():
#         code = "Rush hours"
#     elif dt_value.time() > datetime(1,1,1, 9, 30, 0).time():
#         code = "Middays"
#     else:
#         code = "Rush hours"
#     for i in line: # Adding up the trains for all lines this station services
#         trains += frequency_table.loc[i, code]
#     return trains

In [ ]:
def trains_per_h_conv_better(date_and_line): # Takes datetime and line as args, returns # of trains
    dt_value = date_and_line[0]
    dt_ymd = [dt_value.year, dt_value.month, dt_value.day]
    line = date_and_line[1]
    trains = 0
    time_rem = 240 * 60 # Time remaining in 4 hour segment, in seconds
    tv = (dt_value - datetime(dt_ymd[0], dt_ymd[1], dt_ymd[2])).seconds # Takes datetime value and isolates time, in seconds
    while time_rem > 0: # Will loop back if not all time is used up
        if dt_value.weekday() > 4: # if weekend
            if tv > time_to_s(6, 30): # if after 6:30am
                td = tv - time_to_s(6, 30) # time in interval
                trains += calc_trains(min(td, time_rem), line, "Weekends") # adding to train count
                if time_rem < td: # if 4 hours are up
                    break
                else:
                    time_rem -= td # subtract time used from time remaining
                    tv -= td # dialing back current time to check future conditions
            td = tv # before 6:30am
            trains += calc_trains(min(td, time_rem), line, "Late nights")
            if time_rem < td:
                break
            else:
                time_rem -=td
                tv = 24 * 60 * 60
        if tv > time_to_s(20, 0): # if after 8pm
            td = tv - time_to_s(20, 0)
            trains += calc_trains(min(td, time_rem), line, "Evenings")
            if time_rem < td:
                break
            else:
                time_rem -= td
                tv -= td
        if tv > time_to_s(15, 30): # if after 3:30pm
            td = tv - time_to_s(15, 30)
            trains += calc_trains(min(td, time_rem), line, "Rush hours")
            if time_rem < td:
                break
            else:
                time_rem -= td
                tv -= td
        
        if tv > time_to_s(9, 30): # if after 9:30am
            td = tv - time_to_s(9, 30)
            trains += calc_trains(min(td, time_rem), line, "Middays")
            if time_rem < td:
                break
            else:
                time_rem -= td
                tv -= td
        
        if tv > time_to_s(6, 30): # if after 6:30am
            td = tv - time_to_s(6, 30)
            trains += calc_trains(min(td, time_rem), line, "Rush hours")
            if time_rem < td:
                break
            else:
                time_rem -= td
                tv -= td
        
        td = tv
        trains += calc_trains(min(td, time_rem), line, "Late nights")
        if time_rem < td:
            break
        else:
            time_rem -= td
            tv = 24 * 60 * 60
            dt_value = dt_value - timedelta(days = 1)
    return trains
                
    
def calc_trains(td, line, code): # multiplying by seconds, will need to divide by a large number at end (4 * 60 * 60)
    out = 0
    for i in line:
        out += ((frequency_table.loc[i, code]) * td)/(4*60*60)
    return out

def time_to_s(hours, minutes): # converts hours, minutes to seconds
    return (3600 * hours) + (60 * minutes)

In [ ]:
df_clean['trains_per_h'] = df_clean['trains_per_h'].apply(trains_per_h_conv_better) # Going from (line, datetime) to frequency

In [ ]:
df_clean['avg_wait'] = 0.5 / df_clean['trains_per_h'] # Calculating the average wait at a station (in hours) and the average number of passengers waiting
df_clean['avg_waiting'] = df_clean['new_entries'] * df_clean['avg_wait']
df_clean.sample(10)

In [ ]:
df_clean = df_clean.loc[df_clean['trains_per_h'] > 0] # Getting rid of entries for when no trains would arrive
df_clean.sort_values(by = 'avg_waiting', ascending = False).head(25)

In [ ]:
# Searching for weekly trends by grouping by day of week, time
df_weekly = df_clean.groupby(by = ['station', 'daytime', 'linename', 'trains_per_h'], as_index = False)[['avg_waiting','new_entries', 'total_traffic']].mean().reset_index(drop = True)


In [ ]:
df_weekly = df_weekly.loc[df_weekly['trains_per_h'] > 8].reset_index(drop = True) # Getting rid of times when there are very few trains
df_weekly.sort_values(by= 'avg_waiting', ascending = False).head(20)

In [ ]:
df_weekly.reset_index(drop = True, inplace = True)
df_weekly.reset_index(drop = False, inplace = True)
df_weekly.head()

In [ ]:
# Figure scatter of new_entries vs. trains per h

plt.figure(figsize = (12,9))
plt.scatter(df_weekly['trains_per_h'], df_weekly['new_entries'], s = 5, alpha = 0.4)
plt.xlim(0)
plt.ylim(0)
plt.xlabel("Trains per hour")
plt.ylabel("New entries")
plt.title("Traffic vs. Train Frequency")
plt.savefig(fname = "Figure 3.png")

In [ ]:
# Figure 4: notable stations

df_fig4 = df_weekly.sort_values(by = 'avg_waiting', ascending = False).groupby('station', as_index = False)[['avg_waiting','index']].first().sort_values(by= 'avg_waiting', ascending = False)
df_fig4.head()

In [ ]:
df_weekly.iloc[df_fig4[:20]['index']]

In [ ]:
fig4_keys = [7537, 2776, 2243, 2123, 2109]
fig4 = df_weekly.iloc[fig4_keys]
fig4

In [ ]:
fig, ax1 = plt.subplots()
plt.title("Recommended Stops")
ax1.bar(fig4['station'], fig4['avg_waiting'])
plt.xticks(range(5), labels = ['FLUSHING-MAIN W 7AM', 'ROCKEFELLER STATION W 8PM', 'PENN STATION TU 12PM', '34ST-HUDSON SA 8PM',
                     '34ST HERALD SQ W 8PM'],rotation = 70)
plt.ylabel("Average waiting passengers")
plt.savefig(fname = "Figure 4-1.png")

In [ ]:

fig, ax1 = plt.subplots()

plt.xticks(rotation = 70)
plt.title("Recommended Stops")
color = 'tab:blue'
ax1.set_ylabel("Average waiting passengers", color=color)
ax1.bar(fig4['station'], fig4['avg_waiting'])
#ax1.xticks(range(5), labels = ['FLUSHING-MAIN W 7AM', 'ROCKEFELLER STATION W 7PM', 'PENN STATION TU 9PM', '59ST COLUMBUS SA 7PM',
#                     'UNION SQ TU 9PM'],rotation = 70)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # Shared axis

color = 'tab:green'
ax2.set_ylabel('Trains per hour', color=color)  
ax2.plot(fig4['station'], fig4['trains_per_h'], color=color, linewidth = 5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0)

fig.tight_layout() 
plt.show()
plt.savefig(fname = "Figure 4-2.png")

In [ ]:
#Figure 1: Displaying busiest stations based on total traffic
df_fig1 = df_weekly.sort_values(by = 'total_traffic', ascending = False).groupby('station', as_index = False)[['total_traffic','index']].first().sort_values(by= 'total_traffic', ascending = False).head()['index']
df_fig1 = df_weekly.iloc[df_fig1]

In [ ]:

plt.figure(figsize = (12,9))
fig, ax1 = plt.subplots()

plt.xticks(rotation = 70)
plt.title("The Busiest MTA Stops")

color = 'tab:blue'
ax1.set_ylabel('Total traffic', color=color)
ax1.bar(df_fig1['station'].tolist(), df_fig1['total_traffic'], color=color)

ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # Shared axis

color = 'tab:green'
ax2.set_ylabel('Trains per hour', color=color)  
ax2.plot(df_fig1['station'], df_fig1['trains_per_h'], color=color, linewidth = 5)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0)

#fig.tight_layout() 
plt.show()
plt.savefig('Figure 1-2.png')

In [ ]:

plt.figure(figsize = (12,9))
fig, ax1 = plt.subplots()

plt.xticks(rotation = 70)
plt.title("The Busiest MTA Stops")

color = 'tab:blue'
ax1.set_ylabel('Total traffic', color=color)
ax1.bar(df_fig1['station'].tolist(), df_fig1['total_traffic'], color=color)

ax1.tick_params(axis='y', labelcolor=color)
plt.savefig('Figure 1-1.png')